In [4]:
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from tqdm import tqdm
from src.processing import feature_engineering, preprocessing

df = pl.read_csv(
    "../data/input/train.csv",
    dtypes={
        "stock_id": pl.UInt16,
        "date_id": pl.UInt16,
        "seconds_in_bucket": pl.UInt16,
        "imbalance_size": pl.Float32,
        "imbalance_buy_sell_flag": pl.Int8,
        "reference_price": pl.Float32,
        "matched_size": pl.Float32,
        "far_price": pl.Float32,
        "near_price": pl.Float32,
        "bid_price": pl.Float32,
        "bid_size": pl.Float32,
        "ask_price": pl.Float32,
        "ask_size": pl.Float32,
        "wap": pl.Float32,
        "target": pl.Float32,
        "time_id": pl.UInt32,
    },
)
df.head()

stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
u16,u16,u16,f32,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u32,str
0,0,0,3.1806e6,1,0.999812,1.3380277e7,null,null,0.999812,60651.5,1.000026,8493.030273,1.0,-3.029704,0,"""0_0_0"""
1,0,0,166603.90625,-1,0.999896,1.6422e6,null,null,0.999896,3233.040039,1.00066,20605.089844,1.0,-5.519986,0,"""0_0_1"""
2,0,0,302879.875,-1,0.999561,1.819368e6,null,null,0.999403,37956.0,1.000298,18995.0,1.0,-8.38995,0,"""0_0_2"""
3,0,0,1.1917682e7,-1,1.000171,1.8389746e7,null,null,0.999999,2324.899902,1.000214,479032.40625,1.0,-4.010201,0,"""0_0_3"""
4,0,0,447549.96875,-1,0.999532,1.7860614e7,null,null,0.999394,16485.539062,1.000016,434.100006,1.0,-7.349849,0,"""0_0_4"""


### IndexWAP
- 全銘柄共通で基準となるWAPが割り当てられている
  - これは、株価指数のようなもので、各銘柄のWAPに対して加重平均を取ることで作成されたものである。
  - つまり、銘柄ごとに**重み（係数）**を求めることができる。
- targetは下記の計算式で算出される
  - `stockWAPt`は既知なので、各銘柄の重みがわかれば、`IndexWAPt`が求められる。
  - つまり、targetの予測は、`stockWAPt+60`と`IndexWAPt+60`を予測する問題に帰着する。
$$
\text{Target} = \left( \frac{\text{StockWAP}_{t+60}}{\text{StockWAP}_t} - \frac{\text{IndexWAP}_{t+60}}{\text{IndexWAP}_t} \right) \times 10000
$$


In [5]:
num_stocks = df["stock_id"].n_unique()
num_dates = df["date_id"].n_unique()
num_updates = df["seconds_in_bucket"].n_unique()

print(f"# stocks         : {num_stocks}")
print(f"# dates          : {num_dates}")
print(f"# updates per day: {num_updates}")

stock_returns = np.zeros((num_stocks, num_dates, num_updates))
index_returns = np.zeros((num_stocks, num_dates, num_updates))

# stocks         : 200
# dates          : 481
# updates per day: 55


- `index_return`は下記の値。
  - この値を算出し、stock_idごとのWAPを特徴量として線形回帰を行う。回帰直線の係数が、各銘柄の重みになる。
$$
\frac{\text{IndexWAP}_{t+60}}{\text{IndexWAP}_t}
$$

In [25]:
for (stock_id, date_id), frame in tqdm(
    df.group_by(["stock_id", "date_id"], maintain_order=True),
    total=num_stocks * num_dates,
):
    stock_return = ((frame["wap"] / frame["wap"].shift(6)).shift(-6) - 1) * 10_000
    index_return = stock_return - frame["target"]

    stock_returns[stock_id, date_id] = stock_return.to_numpy()
    index_returns[stock_id, date_id] = index_return.to_numpy()

index_return = index_returns.mean(axis=0)

 99%|█████████▉| 95236/96200 [00:12<00:00, 7547.02it/s]


In [26]:
lr = LinearRegression()
y = index_return.reshape(-1)
X = stock_returns.reshape((num_stocks, -1)).T

mask = ~((np.isnan(y) | np.isnan(X).any(axis=1)))
X, y = X[mask], y[mask]

lr.fit(X, y)

lr.coef_ = lr.coef_.round(3)
lr.intercept_ = 0.0
print("Coef:", lr.coef_)
print("Sum of Coef:", lr.coef_.sum())
print("R2:", r2_score(y, lr.predict(X)))

Coef: [0.004 0.001 0.002 0.006 0.004 0.004 0.002 0.006 0.006 0.002 0.002 0.008
 0.006 0.002 0.008 0.006 0.002 0.006 0.004 0.002 0.004 0.001 0.006 0.004
 0.002 0.002 0.004 0.002 0.004 0.004 0.001 0.001 0.002 0.002 0.006 0.004
 0.004 0.004 0.006 0.002 0.002 0.04  0.002 0.002 0.004 0.04  0.002 0.001
 0.006 0.004 0.004 0.006 0.001 0.004 0.004 0.002 0.006 0.004 0.006 0.004
 0.006 0.004 0.002 0.001 0.002 0.004 0.002 0.008 0.004 0.004 0.002 0.004
 0.006 0.002 0.004 0.004 0.002 0.004 0.004 0.004 0.001 0.002 0.002 0.008
 0.02  0.004 0.006 0.002 0.02  0.002 0.002 0.006 0.004 0.002 0.001 0.02
 0.006 0.001 0.002 0.004 0.001 0.002 0.006 0.006 0.004 0.006 0.001 0.002
 0.004 0.006 0.006 0.001 0.04  0.006 0.002 0.004 0.002 0.002 0.006 0.002
 0.002 0.004 0.006 0.006 0.002 0.002 0.008 0.006 0.004 0.002 0.006 0.002
 0.004 0.006 0.002 0.004 0.001 0.004 0.002 0.004 0.008 0.006 0.008 0.002
 0.004 0.002 0.001 0.004 0.004 0.004 0.006 0.008 0.004 0.001 0.001 0.002
 0.006 0.004 0.001 0.002 0.006 0.004 0.006 0.0